In [1]:
import glob, os
from tqdm import tqdm
from volReader import volFile
dirtoextract = "/data/amd-data/dataraw/MAIA_OCT_20210825"
extracteddir = "/data/amd-data/cera-rpd/MAIA_OCT_20210825"
files_to_extract = glob.glob(os.path.join(dirtoextract,'*.vol'),recursive=True)


In [2]:

def extractFiles():
    for i,line in enumerate(tqdm(files_to_extract)):
        fpath = line.strip('\n')
        path, scan_str = fpath.strip('.vol').rsplit('/',1)
        extractpath = os.path.join(extracteddir,scan_str.replace('_','/'))
        os.makedirs(extractpath,exist_ok=True)
        vol = volFile(fpath)
        preffix = extractpath+'/'+scan_str+'_oct'
        # print('\n'+ preffix)
        vol.renderOCTscans(preffix)


In [3]:
extractFiles()


 94%|█████████▍| 2045/2169 [1:52:23<06:23,  3.10s/it]  /data/amd-data/cera-rpd/detectron2-rpd/datasets/volReader.py:217: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  wholefile["segmentations"] = np.array(segmentations)
100%|██████████| 2169/2169 [1:59:18<00:00,  3.30s/it]


In [6]:
import numpy as np
extracted_files = glob.glob(os.path.join(extracteddir,'**','*.png'),recursive=True)
data = np.array([s.rsplit('/')[-1].rstrip('.png').split('_') for s in extracted_files])
data = np.concatenate([data,np.array(extracted_files)[:,None]],axis=1)

In [8]:
import pandas as pd
df = pd.DataFrame(data,columns=['ptid','eye','time','unknown','slicei','img_path'])
df.loc[:,'slicei']=df.slicei.str.lstrip('oct-').astype(int)
df = df.assign(fold ='CERA_long')
df = df.assign(msk_path = np.nan)
df['volID'] = df.apply(lambda x: '_'.join(x[['ptid','eye','time']].astype(str).values),axis=1)
df = df.astype({'time':'float'})
df.to_parquet('dfCERA_long.parquet')


In [11]:
df.volID.drop_duplicates().shape

(2169,)

In [12]:
from data import rpd_data
import pickle
dat = rpd_data(df,'CERA_long',False)
pickle.dump(dat, open("dfCERA_long.pk", "wb"))


106275it [11:36, 152.56it/s]

Found 106275 images
Found 0 instances
Found 0 too few vertices


In [1]:
import pickle
dat = pickle.load(open('dfCERA_long.pk','rb'))
dat

[{'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-013.png',
  'height': 496,
  'width': 1024,
  'image_id': '101216_R_30_013'},
 {'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-014.png',
  'height': 496,
  'width': 1024,
  'image_id': '101216_R_30_014'},
 {'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-028.png',
  'height': 496,
  'width': 1024,
  'image_id': '101216_R_30_028'},
 {'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-021.png',
  'height': 496,
  'width': 1024,
  'image_id': '101216_R_30_021'},
 {'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-026.png',
  'height': 496,
  'width': 1024,
  'image_id': '101216_R_30_026'},
 {'file_name': '/data/amd-data/cera-rpd/MAIA_OCT_20210825/101216/R/30/4050/101216_R_30_4050_oct-037.png',
  'height': 496,
  'widt